In [ ]:
#EDA

In [ ]:
#모듈

import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
#데이터 확인
raw_data = pd.read_csv("house_price.csv")
raw_data.head()

In [ ]:
#임시책
raw_data = raw_data.fillna("0")

In [ ]:
#데이터 확인
raw_data.columns

In [ ]:
number_col = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea','GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

In [ ]:
category_col = [x for x in raw_data.columns if x not in number_col]

In [ ]:
N=15
wts = category_col[N]
wts = "YearBuilt"
print(wts," {}/{}".format(N, len(category_col)))

In [ ]:
info_category = (
    raw_data.groupby([wts]).size()
            .to_frame()
            .rename(columns = {0 : "size"})
)

print(info_category)

In [ ]:
category = set(raw_data[wts].values)
sample_size = min(set(info_category["size"].values))
condition_price = {}

def make_pivot_full(df):
    for ctg in category :
        condition_price[ctg] = list(df[df[wts].map(lambda x: x == ctg )].sample(sample_size)["SalePrice"])
make_pivot_full(raw_data)

In [ ]:
dat = []
for ctg in category :
    dat.append(pd.DataFrame(condition_price)[ctg])

plt.figure() 
plt.boxplot(dat,1)
plt.xlabel(wts)
plt.ylabel('Race Record')
plt.grid()   
plt.show()

In [ ]:
raw_data.groupby(wts)["SalePrice"].mean().iplot(kind = "bar")

In [ ]:
raw_data.iplot(kind = "scatter", mode = "markers", x= wts, y= "SalePrice")

In [ ]:
#특이값 확인용
raw_data[raw_data["TotRmsAbvGrd"] >=12].head()

In [ ]:
#------------------------------------------------

In [ ]:
import math
raw_data["SalePrice"] = raw_data["SalePrice"].map(lambda x : math.log(x))

In [ ]:
raw_data[wts] = raw_data[wts].map(lambda x : math.log(x))

In [ ]:
#숫자형 변수 EDA
N=12
wts = number_col[N]
print(wts," {}/{}".format(N,len(number_col)))
print("max : ", raw_data.groupby(wts).size().max())
print("min : ", raw_data.groupby(wts).size().min())
raw_data.groupby(wts)["SalePrice"].mean().iplot(kind = "scatter", mode = "markers")

In [ ]:
raw_data

In [ ]:
#-----------------------------------------------

In [ ]:
#숫자형의 카테고리화 EDA
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df

In [ ]:
num_lst = [10] * len(number_col)
category_num_data = dummify_number_cols(raw_data,number_col,num_lst)

In [ ]:
wts = number_col[9]
wts

In [ ]:
print(wts,"/",len(number_col))
print(category_num_data.groupby(wts).size().sort_values(ascending=False)[:5])
print("max : ", raw_data.groupby(wts).size().max())
print("min : ", raw_data.groupby(wts).size().min())
category_num_data.groupby(wts)["SalePrice"].mean().iplot(kind = "bar")

In [ ]:
#--------------------------------------------------

In [ ]:
#카테고리 묶음단위 조정
temp = dummify_number_cols(raw_data,[wts],[100])
print(wts,"/",len(number_col))
print(temp.groupby(wts).size().sort_values(ascending=False)[:5])
print("max : ",raw_data.groupby(wts).size().max())
print("min : ", raw_data.groupby(wts).size().min())
temp.groupby(wts)["SalePrice"].mean().iplot(kind = "bar")